In [1]:
import sys
import re
import traceback
import IPython

# replace the trackback handler to hide paths on server
def exception_hook(*args, **kwargs):
    lines = traceback.format_exception(*sys.exc_info())
    lines = [lines[0]] + lines[2:]
    lines = [re.sub(r'^.+(/cmdy/.+\.py)', r'/path/.../to\1', line) for line in lines]
    lines = [re.sub(r'^.+(/miniconda3/.+\.py)', r'/path/.../to\1', line) for line in lines]
    sys.stderr.write(''.join(lines))
    
IPython.core.interactiveshell.InteractiveShell.showtraceback = exception_hook

# Usage

To run this demo, please clone the whole repository.

## Basic usage

In [2]:
from cmdy import ls

In [3]:
print(ls())

LICENSE
README.md
README.rst
cmdy
demo.ipynb
echo.py
pyproject.toml
pytest.ini
requirements.txt
setup.py
tests



In [4]:
for line in ls().iter():
    print('Got:', line, end='')

Got: LICENSE
Got: README.md
Got: README.rst
Got: cmdy
Got: demo.ipynb
Got: echo.py
Got: pyproject.toml
Got: pytest.ini
Got: requirements.txt
Got: setup.py
Got: tests


## With non-keyword arguments

In [5]:
from cmdy import tar
print(tar("cvf", "/tmp/test.tar", "./cmdy"))

./cmdy/
./cmdy/__init__.py
./cmdy/cmdy_plugin.py
./cmdy/cmdy_util.py
./cmdy/__pycache__/
./cmdy/__pycache__/__init__.cpython-37.pyc
./cmdy/__pycache__/cmdy_plugin.cpython-37.pyc
./cmdy/__pycache__/cmdy_util.cpython-37.pyc



## With keyword arguments

In [6]:
from cmdy import curl
curl("http://duckduckgo.com/", o="/tmp/page.html", silent=True)
# curl http://duckduckgo.com/ -o /tmp/page.html --silent

<CmdyResult: ['curl', 'http://duckduckgo.com/', '-o', '/tmp/page.html', '--silent']>

### Order keyword arguments

In [7]:
curl("http://duckduckgo.com/", "-o", "/tmp/page.html", "--silent")

<CmdyResult: ['curl', 'http://duckduckgo.com/', '-o', '/tmp/page.html', '--silent']>

In [8]:
# or
from diot import OrderedDiot
kwargs = OrderedDiot()
kwargs.silent = True
kwargs.o = '/tmp/page.html'
curl("http://duckduckgo.com/", kwargs)
# You can also use collections.OrderedDict

<CmdyResult: ['curl', 'http://duckduckgo.com/', '--silent', '-o', '/tmp/page.html']>

### Prefix and separator for keyword arguments

In [9]:
from cmdy import bedtools, bcftools
bedtools.intersect(wa=True, wb=True, 
                   a='query.bed', b=['d1.bed', 'd2.bed', 'd3.bed'], 
                   names=['d1', 'd2', 'd3'], sorted=True, 
                   _prefix='-').h().strcmd


'bedtools intersect -wa -wb -a query.bed -b d1.bed d2.bed d3.bed -names d1 d2 d3 -sorted'

In [10]:
# default prefix is auto
bcftools.query(_=['a.vcf', 'b.vcf'], H=True, 
               format='%CHROM\t%POS\t%REF\t%ALT\n').h().strcmd

"bcftools query -H --format '%CHROM\t%POS\t%REF\t%ALT\n' a.vcf b.vcf"

In [11]:
ls(l=True, block_size='KB', _sep='auto').h().cmd

['ls', '-l', '--block-size=KB']

### Mixed combinations of prefices and separators in one command

In [12]:
from cmdy import java
# Note this is just an example for old verion picard. 
# Picard is changing it's style

picard = java(jar='picard.jar', _prefix='', _sep='=', _sub=True)
c = picard.SortSam(I='input.bam', O='sorted.bam', 
               SORTED_ORDER='coordinate',
               _prefix='', _sep='=', _deform=None).h
print(c.cmd)

# same as the above
java({'jar': 'picard.jar', '_prefix': '-', '_sep': ' '}, 
     'SortSam', I='input.bam', O='sorted.bam', 
     SORTED_ORDER='coordinate', _prefix='', _sep='=', _deform=None).h().cmd

# _deform prevents SORTED_ORDER to be deformed to SORTED-ORDER

['java', 'jar=picard.jar', 'SortSam', 'I=input.bam', 'O=sorted.bam', 'SORTED_ORDER=coordinate']


['java',
 'jar=picard.jar',
 'SortSam',
 'I=input.bam',
 'O=sorted.bam',
 'SORTED_ORDER=coordinate']

### Subcommands

In [13]:
from cmdy import git
git.branch(v=True).fg

* async  d6c2a15 FIx baking revert and add new baking way.
  master 92a6209 0.2.2


<CmdyResult: ['git', 'branch', '-v']>

In [14]:
# What if I have separate arguments for main and sub-command?
#import cmdy
#cmdy.git(git_dir='.', _sub=True).branch(v=True)
git(git_dir='.', _sub=True).branch(v=True).h

<CmdyHolding: ['git', '--git-dir', '.', 'branch', '-v']>

### Duplicated keys for list arguments:

In [15]:
from cmdy import sort
print(sort(k=['1,1', '2,2'], t='_', _='./.editorconfig', _dupkey=True))
# sort -k 1,1 -k 2,2 ./.editorconfig


[*]
end_of_line          = lf
indent_size          = 4
indent_style         = tab
insert_final_newline = true
root = true
tab_width            = 4



## Return code and exception

In [16]:
from cmdy import x
x()

Traceback (most recent call last):
  File "<ipython-input-16-092cc5b72e61>", line 2, in <module>
    x()
/path/.../to/cmdy/__init__.py", line 146, in __call__
    ready_cfgargs, ready_popenargs, _will())
/path/.../to/cmdy/__init__.py", line 201, in __new__
    result = holding.run()
/path/.../to/cmdy/__init__.py", line 854, in run
    return orig_run(self, wait)
/path/.../to/cmdy/__init__.py", line 717, in run
    return orig_run(self, wait)
/path/.../to/cmdy/__init__.py", line 327, in run
    ret = CmdyResult(self._run(), self)
/path/.../to/cmdy/__init__.py", line 271, in _run
    raise CmdyExecNotFoundError(str(fnfe)) from None
cmdy.cmdy_util.CmdyExecNotFoundError: [Errno 2] No such file or directory: 'x': 'x'


In [17]:
from cmdy import ls
ls('non-existing-file')

Traceback (most recent call last):
  File "<ipython-input-17-132683fc2227>", line 2, in <module>
    ls('non-existing-file')
/path/.../to/cmdy/__init__.py", line 146, in __call__
    ready_cfgargs, ready_popenargs, _will())
/path/.../to/cmdy/__init__.py", line 204, in __new__
    return result.wait()
/path/.../to/cmdy/__init__.py", line 407, in wait
    raise CmdyReturnCodeError(self)
cmdy.cmdy_util.CmdyReturnCodeError: Unexpected RETURN CODE 2, expecting: [0]

  [   PID] 167164

  [   CMD] ['ls non-existing-file']

  [STDOUT] 

  [STDERR] ls: cannot access non-existing-file: No such file or directory



### Don't raise exception but store the return code

In [18]:
from cmdy import ls
result = ls('non-existing-file', _raise=False)
result.rc

2

### Tolerance on return code

In [19]:
from cmdy import ls
ls('non-existing-file', _okcode='0,2').rc # or [0,2]

2

### Timeouts

In [20]:
from cmdy import sleep
sleep(3, _timeout=1)

Traceback (most recent call last):
  File "<ipython-input-20-47b0ec7af55f>", line 2, in <module>
    sleep(3, _timeout=1)
/path/.../to/cmdy/__init__.py", line 146, in __call__
    ready_cfgargs, ready_popenargs, _will())
/path/.../to/cmdy/__init__.py", line 204, in __new__
    return result.wait()
/path/.../to/cmdy/__init__.py", line 404, in wait
    ) from None
cmdy.cmdy_util.CmdyTimeoutError: Timeout after 1 seconds.


## Redirections

In [21]:
from cmdy import cat
cat('./pytest.ini').redirect > '/tmp/pytest.ini'
print(cat('/tmp/pytest.ini'))

[pytest]
addopts = -vv --cov=cmdy --cov-report xml:.coverage.xml --cov-report term-missing
console_output_style = progress
junit_family=xunit1



### Appending

In [22]:
# r short for redirect
cat('./pytest.ini').r >> '/tmp/pytest.ini'
print(cat('/tmp/pytest.ini'))

[pytest]
addopts = -vv --cov=cmdy --cov-report xml:.coverage.xml --cov-report term-missing
console_output_style = progress
junit_family=xunit1
[pytest]
addopts = -vv --cov=cmdy --cov-report xml:.coverage.xml --cov-report term-missing
console_output_style = progress
junit_family=xunit1



### Redirecting to a file handler

In [23]:
f = open('/tmp/pytest.ini', 'w')
# executing fails to detect future action in with block with ipython
# but feel free to write with block in regular python
cat('./pytest.ini').r > f
f.close()
print(cat('/tmp/pytest.ini'))

[pytest]
addopts = -vv --cov=cmdy --cov-report xml:.coverage.xml --cov-report term-missing
console_output_style = progress
junit_family=xunit1



### STDIN, STDOUT and/or STDERR redirections

In [24]:
from cmdy import STDIN, STDOUT, STDERR, DEVNULL

c = cat().r(STDIN) < '/tmp/pytest.ini'
print(c)

[pytest]
addopts = -vv --cov=cmdy --cov-report xml:.coverage.xml --cov-report term-missing
console_output_style = progress
junit_family=xunit1



In [25]:
# Mixed
c = cat().r(STDIN, STDOUT) ^ '/tmp/pytest.ini' > DEVNULL
# we can't fetch result from a redirected pipe
print(c.stdout)

# Why not '<' for STDIN?
# Because the priority of the operator is not in sequential order.
# We can use < for STDIN, but we need to ensure it runs first
c = (cat().r(STDIN, STDOUT) < '/tmp/pytest.ini') > DEVNULL
print(c.stdout)

# A simple rule for multiple redirections to always use ">" in the last place

None
None


In [26]:
# Redirect stderr to stdout
from cmdy import bash
c = bash(c="cat 1>&2").r(STDIN, STDERR) ^ '/tmp/pytest.ini' > STDOUT
print(c.stdout)

[pytest]
addopts = -vv --cov=cmdy --cov-report xml:.coverage.xml --cov-report term-missing
console_output_style = progress
junit_family=xunit1



In [27]:
# All at the same time
c = bash(c="cat 1>&2").r(STDIN, STDOUT, STDERR) ^ '/tmp/pytest.ini' ^ DEVNULL > STDOUT
print(c.stdout)
print(c.stderr)

None
None


## Pipings

In [28]:
from cmdy import grep
c = ls().p | grep('README')
print(c)

README.md
README.rst



In [29]:
# p short for pipe
c = ls().p | grep('README').p | grep('md')
print(c)
print(c.piped_strcmds)

README.md

['ls', 'grep README', 'grep md']


In [30]:
from cmdy import _CMDY_EVENT
# !!! Pipings should be consumed immediately!
# !!! DO NOT do this
ls().p
ls() # <- Will not run as expected
# All commands will be locked as holding until pipings are consumed
_CMDY_EVENT.clear()
print(ls())

# See Advanced/Holdings if you want to hold a piping command for a while

LICENSE
README.md
README.rst
cmdy
demo.ipynb
echo.py
pyproject.toml
pytest.ini
requirements.txt
setup.py
tests



## Running command in foreground

In [31]:
ls().fg

LICENSE
README.md
README.rst
cmdy
demo.ipynb
echo.py
pyproject.toml
pytest.ini
requirements.txt
setup.py
tests


<CmdyResult: ['ls']>

In [32]:
from cmdy import tail
tail('/tmp/pytest.ini', f=True, _timeout=3).fg
# This mimics the `tail -f` program
# You will see the content comes out one after another
# and then program hangs


[pytest]
addopts = -vv --cov=cmdy --cov-report xml:.coverage.xml --cov-report term-missing
console_output_style = progress
junit_family=xunit1
Traceback (most recent call last):
  File "<ipython-input-32-75e616da45cd>", line 2, in <module>
    tail('/tmp/pytest.ini', f=True, _timeout=3).fg
/path/.../to/cmdy/cmdy_util.py", line 166, in wrapper
    return func(self)
/path/.../to/cmdy/cmdy_plugin.py", line 113, in wrapper
    return func(self, *args, **kwargs)
/path/.../to/cmdy/__init__.py", line 708, in foreground
    return self.run()
/path/.../to/cmdy/__init__.py", line 854, in run
    return orig_run(self, wait)
/path/.../to/cmdy/__init__.py", line 735, in run
    ret, self.data.foreground.poll_interval
/path/.../to/miniconda3/lib/python3.7/site-packages/curio/kernel.py", line 826, in run
    return kernel.run(corofunc, *args)
/path/.../to/miniconda3/lib/python3.7/site-packages/curio/kernel.py", line 173, in run
    raise ret_exc
/path/.../to/miniconda3/lib/python3.7/site-packages/cur

In [33]:
# You also write an `echo-like` program easily
# 
# This will not run here
# !!! NOT RUN
# Save it to a file and run with python interpreter
# See echo.py

# from cmdy import cat
# cat().fg(stdin=True)

## Iterating on output

In [34]:
for line in ls().iter():
    print(line, end='')

LICENSE
README.md
README.rst
cmdy
demo.ipynb
echo.py
pyproject.toml
pytest.ini
requirements.txt
setup.py
tests


### Iterating on stderr

In [35]:
for line in bash(c="cat /tmp/pytest.ini 1>&2").iter(STDERR):
    print(line, end='')

[pytest]
addopts = -vv --cov=cmdy --cov-report xml:.coverage.xml --cov-report term-missing
console_output_style = progress
junit_family=xunit1


### Getting live output

In [36]:
# Like we did for `tail -f` program
# This time, we can do something with each output line

# Let's use a thread to write content to a file
# And we try to get the live contents using cmdy
import time
from threading import Thread
def live_write(file, n):
    
    with open(file, 'w', buffering=1) as f:
        # Let's write something every half second
        for i in range(n):
            f.write(str(i) + '\n')
            time.sleep(.5)
            
test_file = '/tmp/tail-f.txt'
Thread(target=live_write, args=(test_file, 10)).start()

from cmdy import tail

tail_iter = tail(f=True, _=test_file).iter()

for line in tail_iter:
    # Do whatever you want with the line
    print('We got:', line, end='')
    if line.strip() == '8':
        break
        
# make sure thread ends
time.sleep(2)

We got: 0
We got: 1
We got: 2
We got: 3
We got: 4
We got: 5
We got: 6
We got: 7
We got: 8


In [37]:
# What about timeout?

# Of course you can use a timer to check inside the loop
# You can also set a timeout for each fetch

# Terminate after 10 queries

Thread(target=live_write, args=(test_file, 10)).start()

from cmdy import tail

tail_iter = tail(f=True, _=test_file).iter()

for i in range(10):
    print('We got:', tail_iter.next(timeout=1), end='')
    

We got: 0
We got: 1
We got: 2
We got: 3
We got: 4
We got: 5
We got: 6
We got: 7
We got: 8
We got: 9


## Advanced

### Baking `Cmdy` object

Sometimes, you may want to run the same program a couple of times, with the same set of arguments or configurations, and you don't want to type those arguments every time, then you can bake the `Cmdy` object with that same arguments or configurations.

For example, if you want to run `ls` as `ls -l` all the time:

In [38]:
from cmdy import ls
ll = ls.bake(l=True)
print(ll().h.cmd)
print(ll(a=True).h.cmd)
# I don't want the l flag for some commands occasionally
print(ll(l=False).h.cmd)

# Bake a baked command
lla = ll.bake(a=True)
print(lla().h.cmd)

['ls', '-l']
['ls', '-l', '-a']
['ls']
['ls', '-l', '-a']


In [39]:
# I know git is always gonna run with subcommand
git = git.bake(_sub=True)
print(git(git_dir='.').branch(v=True).h)
print(git().status().h)

<CmdyHolding: ['git', '--git-dir', '.', 'branch', '-v']>
<CmdyHolding: ['git', 'status']>


In [40]:
# What if I have a subcommand call 'bake'?
from cmdy import git, CmdyActionError

print(git.branch().h.cmd)
try:
    git.bake().h.cmd
except CmdyActionError as ex:
    print(ex)

# run the git with _sub
print(git(_sub=True).bake().h.cmd)

['git', 'branch']
['git', 'bake']


### Baking the whole module

In [41]:
import cmdy
sh = cmdy(version=True)
# anything under sh directly will be supposed to have subcommand
from sh import git, gcc
print(git().h)
print(gcc().h)

<CmdyHolding: ['git', '--version']>
<CmdyHolding: ['gcc', '--version']>


Note that module baking is deep copying, except the exception classes and some utils.
This means, you would expect following behavior:

In [42]:
import cmdy
from cmdy import CmdyHolding, CmdyExecNotFoundError

sh = cmdy()

c = sh.echo().h
print(type(c))
print(isinstance(c, CmdyHolding)) # False
print(isinstance(c, sh.CmdyHolding)) # True

try:
    sh.notexisting()
except CmdyExecNotFoundError:
    # we can catch it, as CmdyExecNotFoundError is sh.CmdyExecNotFoundError
    print('Catched!')


<class 'cmdy.CmdyHolding'>
False
True
Catched!


### Holding objects

You may have noticed that we have a couple of examples above with a final call `.h` or `.h()`, which is holding the command from running.

You can do that, too, if you have multiple operations

In [43]:
print(ls().h)

# however, you can hold after some actions
ls().fg.h

<CmdyHolding: ['ls']>
Traceback (most recent call last):
  File "<ipython-input-43-f1aed6d50b3b>", line 4, in <module>
    ls().fg.h
/path/.../to/cmdy/cmdy_util.py", line 166, in wrapper
    return func(self)
/path/.../to/cmdy/cmdy_plugin.py", line 108, in wrapper
    raise CmdyActionError("Action taken after a final action.")
cmdy.cmdy_util.CmdyActionError: Action taken after a final action.


In [44]:
# Once a command is on hold (by .h, .hold, .h() or .hold())
# You have to explictly call run() to set the command running
from time import time
tic = time()
c = sleep(2).h
print(f'Time elapsed: {time() - tic:.3f} s')
# not running even with fg
c.fg
print(f'Time elapsed: {time() - tic:.3f} s')
c.run()
print(f'Time elapsed: {time() - tic:.3f} s')

Time elapsed: 0.022 s
Time elapsed: 0.034 s
Time elapsed: 2.043 s


### Reuse of command

In [45]:
# After you set a command running,
# you can retrieve the holding object,
# and reuse it
from cmdy import ls
c = ls().fg
# nothing will be produced
c.holding.reset().r > DEVNULL

LICENSE
README.md
README.rst
cmdy
demo.ipynb
echo.py
pyproject.toml
pytest.ini
requirements.txt
setup.py
tests


<CmdyResult: ['ls']>

### Async mode

In [46]:
import curio
from cmdy import ls
a = ls().a # async command is never blocking!

async def main():
    async for line in a:
        print(line, end='')

curio.run(main())

LICENSE
README.md
README.rst
cmdy
demo.ipynb
echo.py
pyproject.toml
pytest.ini
requirements.txt
setup.py
tests


### Extending `cmdy`

All those actions for holding/result objects were implemented internally as plugins. You can right your own plugins, too.

A plugin has to be defined as a class and then instantiated. 

**There are 6 APIs for developing a plugin for `cmdy`**

- `cmdy_plugin`: A decorator for the plugin class
- `cmdy_plugin_hold_then`: A decorator to decorate methods in the plugin class, which define actions after a holding object. Arguments:
  - `alias`: The alias of this action (e.g. `r/redir` for `redirect`)
  - `final`: Whether this is a final action, meaning no other actions should be followed
  - `prop`: Whether this action can be called as a property
  - `hold_right`: Should I put right following action on hold? This is useful when we have connectors which then can set the command running. (e.g `>` for redirect and `|` for pipe)
- `cmdy_plugin_run_then`: A decorator to decorate methods in the plugin class, which define actions after a sync result object. Arguments are similar as `cmdy_plugin_hold_then` except that `prop` and `hold_right` are not avaialbe.
- `cmdy_plugin_async_run_then`: Async verion of `cmdy_plugin_run_then`
- `cmdy_plugin_add_method`: A decorator to decorate methods in the plugin class, which add methods to the `CmdyHolding`, `CmdyResult` or `CmdyAsyncResult` class. `cls` is the only argument that specifies which class we are hacking.
- `cmdy_plugin_add_property`: Property version of `cmdy_plugin_add_method`

**Notes on name conflicts:**

If we need to add the methods to multiple classes in the plugin with the same name, you can define a different name with extra underscore suffix(es).

**Notes on module baking:**

- As we mentioned before, `cmdy` module baking are deep copying. So when we can pass the class name instead of the class itself (which you may be not sure which one to use, the orginal one or the one from the baking module) to the `add_method` and `add_property` hooks.
- Plugin enable and disable only take effect within the same module. For example:

    ```python
    import cmdy
    from cmdy import CMDY_PLUGIN_FG
    sh = cmdy()
    # only affects cmdy not sh
    CMDY_PLUGIN_FG.disable() 
    # to disable this plugin for sh as well:
    sh.CMDY_PLUGIN_FG.disable()
    ```



In [47]:
# An example to define a plugin
from cmdy import (cmdy_plugin, 
                  cmdy_plugin_hold_then, 
                  cmdy_plugin_add_method, 
                  ls, 
                  CmdyActionError)

@cmdy_plugin
class MyPlugin:
    @cmdy_plugin_add_method("CmdyHolding")
    def say_hello(self):
        return 'Hello world!'

    @cmdy_plugin_hold_then('hello')
    def helloworld(self):
        print(self.say_hello())
        # keep chaining
        return self

myplugin = MyPlugin()

# command will never run, 
# because we didn't do self.run() in helloworld(self)
ls().helloworld() 
# property calls enabled by default
ls().helloworld
# we have alias
ls().hello


Hello world!
Hello world!
Hello world!


<CmdyHolding: ['ls']>